In [2]:
import time
import numpy as np
import pandas as pd
import gym
import sqlite3
from binance.client import Client
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
import pymysql
from sqlalchemy import create_engine

In [32]:
# Binance API credentials
API_KEY='9nYgDSuxQds12SMtmav2saEFEYvCy5qXMfKIxFH7ehmWBEP2Q0Ehy8a3tkyUImdL'
API_SECRET='1eTmL7RhbCJTycfBpIHLjVeLLvVPEkHeag8qzve1pRm9vOOx8eU9ugj9Wa6zKoWe'

# Initialize Binance client
client = Client(API_KEY, API_SECRET)

In [ ]:

# MySQL Connection Setup
DB_USER = "root"
DB_PASSWORD = "redSQL5!"
DB_HOST = "localhost"
DB_NAME = "binance_data"

engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}")

# Trading pairs
SYMBOLS = ["BTCUSDT", "ETHUSDT", "BNBUSDT", "SOLUSDT", "XRPUSDT"]
INTERVAL = Client.KLINE_INTERVAL_1MINUTE

# Function to fetch live market data for multiple pairs
def get_binance_data():
    all_data = []
    try:
        for symbol in SYMBOLS:
            klines = client.get_klines(symbol=symbol, interval=INTERVAL, limit=1)
            for kline in klines:
                data = {
                    "symbol": symbol,
                    "timestamp": pd.to_datetime(kline[0], unit='ms'),
                    "open_price": float(kline[1]),
                    "high_price": float(kline[2]),
                    "low_price": float(kline[3]),
                    "close_price": float(kline[4]),
                    "volume": float(kline[5])
                }
                all_data.append(data)
        return all_data
    except Exception as e:
        print(f"Error fetching Binance data: {e}")
        return None

# Function to insert data into MySQL
def insert_into_mysql(data_list):
    try:
        if data_list:
            df = pd.DataFrame(data_list)
            df.to_sql("market_data", con=engine, if_exists="append", index=False)
            print(f"Inserted {len(data_list)} records into MySQL.")
    except Exception as e:
        print(f"Error inserting data into MySQL: {e}")
        
# Global flag to control loop execution
RUNNING = True

# Main loop to continuously fetch & insert data until manually stopped
if __name__ == "__main__":
    try:
        print("Fetching and inserting data. Press Ctrl+C to stop.")
        while RUNNING:
            market_data = get_binance_data()
            if market_data:
                insert_into_mysql(market_data)
            time.sleep(60)  # Fetch new data every minute
    except KeyboardInterrupt:
        RUNNING = False
        print("\nData fetching stopped by user.")

In [ ]:
# Database Connection
DB_PATH = "binance_data.db"
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# Trading parameters
SYMBOLS = ["BTCUSDT", "ETHUSDT", "BNBUSDT", "SOLUSDT", "XRPUSDT"]
INTERVAL = Client.KLINE_INTERVAL_5MINUTE
TRADE_QUANTITY = 0.001  

# Market Regime Detection
class MarketRegime:
    @staticmethod
    def detect_regime(df):
        df['returns'] = df['close_price'].pct_change()
        df['volatility'] = df['returns'].rolling(20).std()
        df['momentum'] = df['close_price'].diff(10)
        
        if df['volatility'].iloc[-1] > df['volatility'].rolling(50).mean().iloc[-1]:
            return "trending"
        elif df['momentum'].abs().iloc[-1] < df['momentum'].rolling(50).mean().iloc[-1]:
            return "mean-reverting"
        elif df['volatility'].iloc[-1] > df['volatility'].rolling(20).mean().iloc[-1]:
            return "volatile"
        else:
            return "low-liquidity"

# Fetch market data from Binance
def fetch_binance_data():
    all_data = []
    for symbol in SYMBOLS:
        klines = client.get_klines(symbol=symbol, interval=INTERVAL, limit=1)
        for kline in klines:
            data = {
                "symbol": symbol,
                "timestamp": pd.to_datetime(kline[0], unit='ms'),
                "open_price": float(kline[1]),
                "high_price": float(kline[2]),
                "low_price": float(kline[3]),
                "close_price": float(kline[4]),
                "volume": float(kline[5])
            }
            all_data.append(data)
    return pd.DataFrame(all_data)

# Fetch historical market data from database
def fetch_historical_data():
    query = "SELECT * FROM market_data WHERE symbol IN ('BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'SOLUSDT', 'XRPUSDT') ORDER BY timestamp DESC LIMIT 100"
    df = pd.read_sql(query, conn)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    return df

# Reinforcement Learning Environment
class TradingEnv(gym.Env):
    def __init__(self):
        super(TradingEnv, self).__init__()
        self.action_space = gym.spaces.Discrete(3)  # 0: Hold, 1: Buy, 2: Sell
        self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(6,), dtype=np.float32)
        self.balance = 10000
        self.position = 0
        self.last_price = 0
        self.df = self.load_data()
        self.current_step = 50
    
    def load_data(self):
        try:
            df = fetch_historical_data()
            if df.empty:
                print("No historical data found. Fetching live data...")
                df = fetch_binance_data()
        except Exception as e:
            print(f"Error fetching data from database: {e}. Fetching live data...")
            df = fetch_binance_data()
        return df
    
    def reset(self):
        self.current_step = 50
        self.balance = 10000
        self.position = 0
        self.last_price = self.df.iloc[self.current_step]['close_price']
        return self._get_observation()
    
    def _get_observation(self):
        row = self.df.iloc[self.current_step]
        return np.array([row['close_price'], row['high_price'], row['low_price'], self.position, self.balance, row['volume']], dtype=np.float32)
    
    def step(self, action):
        self.current_step += 1
        if self.current_step >= len(self.df) - 1:
            done = True
            return self._get_observation(), 0, done, {}
        
        price = self.df.iloc[self.current_step]['close_price']
        reward = 0
        market_regime = MarketRegime.detect_regime(self.df)
        
        if market_regime == "trending":
            if action == 1 and price > self.df['open_price'].iloc[self.current_step]:
                order = client.order_market_buy(symbol=SYMBOLS[0], quantity=TRADE_QUANTITY)
                self.position += TRADE_QUANTITY
                self.balance -= price * TRADE_QUANTITY
                print(f"BUY Order Executed: {order}, New Balance: {self.balance}")
                reward = 1
            elif action == 2 and price < self.df['open_price'].iloc[self.current_step]:
                order = client.order_market_sell(symbol=SYMBOLS[0], quantity=TRADE_QUANTITY)
                self.balance += price * TRADE_QUANTITY
                self.position -= TRADE_QUANTITY
                print(f"SELL Order Executed: {order}, New Balance: {self.balance}")
                reward = 1
        
        self.last_price = price
        done = False
        return self._get_observation(), reward, done, {}

# Train the model
env = DummyVecEnv([lambda: TradingEnv()])
model = PPO("MlpPolicy", env, verbose=1)

print("Training model...")
model.learn(total_timesteps=10000)

# Save trained model
model.save("reinforcement_trading_model")

print("Model training complete!")

In [24]:
client.get_account()

{'makerCommission': 10,
 'takerCommission': 10,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00100000',
  'taker': '0.00100000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': True,
 'canDeposit': True,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'preventSor': False,
 'updateTime': 1737537858859,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BTC', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'NEO', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'BNB', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'QTUM', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'EOS', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'SNT', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'BNT', 'free': '0.00000000', 'locke

In [17]:
df5=pd.read_csv("historical_market_data.csv")

In [29]:
import time
import pandas as pd
from binance.client import Client
import os

# Define trading pairs and interval
SYMBOLS = ["BTCUSDT", "ETHUSDT", "BNBUSDT", "SOLUSDT", "XRPUSDT"]
INTERVAL = Client.KLINE_INTERVAL_5MINUTE  # Daily data

# Define the start date (10 years ago)
start_str = "2014-01-01"

# Function to fetch historical data
def fetch_historical_data(symbol, interval, start_date):
    print(f"Fetching data for {symbol}...")
    klines = client.get_historical_klines(symbol, interval, start_date)
    data = []
    
    for kline in klines:
        data.append({
            "symbol": symbol,
            "timestamp": pd.to_datetime(kline[0], unit="ms"),
            "open_price": float(kline[1]),
            "high_price": float(kline[2]),
            "low_price": float(kline[3]),
            "close_price": float(kline[4]),
            "volume": float(kline[5])
        })
    
    return pd.DataFrame(data)

# Fetch and consolidate data
all_data = []
for symbol in SYMBOLS:
    df = fetch_historical_data(symbol, INTERVAL, start_str)
    all_data.append(df)
    time.sleep(1)  # Sleep to avoid API rate limits

# Combine all data
historical_data = pd.concat(all_data, ignore_index=True)


csv_filename = "historical_market_data.csv"
historical_data.to_csv(csv_filename, index=False)


Fetching data for BTCUSDT...
Fetching data for ETHUSDT...
Fetching data for BNBUSDT...
Fetching data for SOLUSDT...
Fetching data for XRPUSDT...
